In [2]:
# Import all the necessary modules
import os
import sys
import os, sys
# from .../research/notebooks -> go up two levels to repo root
repo_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mtick
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score 
import pandas_datareader as pdr
import math
import datetime
import itertools
import ast
import yfinance as yf
import seaborn as sn
from IPython.display import display, HTML
from strategy_signal.trend_following_signal import (
    apply_jupyter_fullscreen_css, get_trend_donchian_signal_for_portfolio_with_rolling_r_sqr_vol_of_vol
)
from portfolio.strategy_performance import (calculate_sharpe_ratio, calculate_calmar_ratio, calculate_CAGR, calculate_risk_and_performance_metrics,
                                          calculate_compounded_cumulative_returns, estimate_fee_per_trade, rolling_sharpe_ratio)
from utils import coinbase_utils as cn
from portfolio import strategy_performance as perf
from sizing import position_sizing_binary_utils as size_bin
from sizing import position_sizing_continuous_utils as size_cont
from strategy_signal import trend_following_signal as tf
%matplotlib inline

In [3]:
import importlib
importlib.reload(cn)
importlib.reload(perf)
importlib.reload(tf)
importlib.reload(size_bin)
importlib.reload(size_cont)

<module 'sizing.position_sizing_continuous_utils' from '/Users/adheerchauhan/Documents/git/trend_following/sizing/position_sizing_continuous_utils.py'>

In [6]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('Display.max_rows', None)
pd.set_option('Display.max_columns',None)
apply_jupyter_fullscreen_css()

In [8]:
from pathlib import Path
import yaml

def load_prod_strategy_config(strategy_version='v0.1.0'):

    nb_cwd = Path.cwd()  # git/trend_following/research/notebooks
    config_path = (
        nb_cwd.parents[1]                    # -> git/trend_following
        / "live_strategy"
        / f"trend_following_strategy_{strategy_version}-live"
        / "config"
        / f"trend_strategy_config_{strategy_version}.yaml"
    )
    
    print(config_path)            # sanity check
    print(config_path.exists())   # should be True
    
    with open(config_path, "r") as f:
        cfg = yaml.safe_load(f)

    return cfg

In [10]:
cfg = load_prod_strategy_config()

/Users/adheerchauhan/Documents/git/trend_following/live_strategy/trend_following_strategy_v0.1.0-live/config/trend_strategy_config_v0.1.0.yaml
True


In [12]:
cfg

{'run': {'start_date': '2022-04-01',
  'end_date': '2025-07-31',
  'use_specific_start_date': True,
  'signal_start_date': '2022-04-01',
  'warmup_days': 300,
  'long_only': True,
  'annual_trading_days': 365,
  'initial_capital': 15000},
 'universe': {'tickers': ['BTC-USD',
   'ETH-USD',
   'SOL-USD',
   'ADA-USD',
   'AVAX-USD']},
 'data': {'use_coinbase_data': True,
  'use_saved_files': True,
  'saved_file_end_date': '2025-07-31',
  'price_or_returns_calc': 'price',
  'moving_avg_type': 'exponential'},
 'signals': {'moving_average': {'fast_mavg': 20,
   'slow_mavg': 200,
   'mavg_stepsize': 8,
   'mavg_z_score_window': 126},
  'donchian': {'entry_rolling_donchian_window': 56,
   'exit_rolling_donchian_window': 28,
   'use_donchian_exit_gate': False},
  'weighting': {'ma_crossover_signal_weight': 0.9,
   'donchian_signal_weight': 0.1,
   'weighted_signal_ewm_window': 4},
  'activation': {'use_activation': False,
   'tanh_activation_constant_dict': None},
  'filters': {'rolling_r2': {

In [14]:
cfg['signals']

{'moving_average': {'fast_mavg': 20,
  'slow_mavg': 200,
  'mavg_stepsize': 8,
  'mavg_z_score_window': 126},
 'donchian': {'entry_rolling_donchian_window': 56,
  'exit_rolling_donchian_window': 28,
  'use_donchian_exit_gate': False},
 'weighting': {'ma_crossover_signal_weight': 0.9,
  'donchian_signal_weight': 0.1,
  'weighted_signal_ewm_window': 4},
 'activation': {'use_activation': False,
  'tanh_activation_constant_dict': None},
 'filters': {'rolling_r2': {'rolling_r2_window': 100,
   'lower_r_sqr_limit': 0.45,
   'upper_r_sqr_limit': 0.9,
   'r2_smooth_window': 3,
   'r2_confirm_days': 0,
   'r2_strong_threshold': 0.75},
  'vol_of_vol': {'log_std_window': 14,
   'coef_of_variation_window': 20,
   'vol_of_vol_z_score_window': 126,
   'vol_of_vol_p_min': 0.1}}}

In [ ]:
def get_strategy_trend_signal(cfg):

    end_date = datetime.datetime.now().date()
    start_date = end_date - pd.Timedelta(days=cfg['run']['warmup_days'])
    start_date=start_date, end_date=end_date



    # Build kwargs directly from cfg sections
    sig_kwargs = {
        # run / universe / data
        "ticker_list": cfg["universe"]["tickers"],
        "moving_avg_type": cfg["data"]["moving_avg_type"],
        "price_or_returns_calc": cfg["data"]["price_or_returns_calc"],
        "use_coinbase_data": cfg["data"]["use_coinbase_data"],
        "use_saved_files": False,
        "saved_file_end_date": cfg["data"]["saved_file_end_date"],  # convert to date if your fn expects it
    
        # signals → moving average
        "fast_mavg": cfg["signals"]["moving_average"]["fast_mavg"],
        "slow_mavg": cfg["signals"]["moving_average"]["slow_mavg"],
        "mavg_stepsize": cfg["signals"]["moving_average"]["mavg_stepsize"],
        "mavg_z_score_window": cfg["signals"]["moving_average"]["mavg_z_score_window"],
    
        # signals → donchian
        "entry_rolling_donchian_window": cfg["signals"]["donchian"]["entry_rolling_donchian_window"],
        "exit_rolling_donchian_window": cfg["signals"]["donchian"]["exit_rolling_donchian_window"],
        "use_donchian_exit_gate": cfg["signals"]["donchian"]["use_donchian_exit_gate"],
    
        # signals → weighting
        "ma_crossover_signal_weight": cfg["signals"]["weighting"]["ma_crossover_signal_weight"],
        "donchian_signal_weight": cfg["signals"]["weighting"]["donchian_signal_weight"],
        "weighted_signal_ewm_window": cfg["signals"]["weighting"]["weighted_signal_ewm_window"],
    
        # signals → activation
        "use_activation": cfg["signals"]["activation"]["use_activation"],
        "tanh_activation_constant_dict": cfg["signals"]["activation"]["tanh_activation_constant_dict"],
    
        # signals → filters → rolling_r2
        "rolling_r2_window": cfg["signals"]["filters"]["rolling_r2"]["rolling_r2_window"],
        "lower_r_sqr_limit": cfg["signals"]["filters"]["rolling_r2"]["lower_r_sqr_limit"],
        "upper_r_sqr_limit": cfg["signals"]["filters"]["rolling_r2"]["upper_r_sqr_limit"],
        "r2_smooth_window": cfg["signals"]["filters"]["rolling_r2"]["r2_smooth_window"],
        "r2_confirm_days": cfg["signals"]["filters"]["rolling_r2"]["r2_confirm_days"],
        "r2_strong_threshold": cfg["signals"]["filters"]["rolling_r2"]["r2_strong_threshold"],
    
        # signals → filters → vol_of_vol
        "log_std_window": cfg["signals"]["filters"]["vol_of_vol"]["log_std_window"],
        "coef_of_variation_window": cfg["signals"]["filters"]["vol_of_vol"]["coef_of_variation_window"],
        "vol_of_vol_z_score_window": cfg["signals"]["filters"]["vol_of_vol"]["vol_of_vol_z_score_window"],
        "vol_of_vol_p_min": cfg["signals"]["filters"]["vol_of_vol"]["vol_of_vol_p_min"],
    
        # run
        "long_only": cfg["run"]["long_only"],
    }
    
    df_trend = get_trend_donchian_signal_for_portfolio_with_rolling_r_sqr_vol_of_vol(**sig_kwargs)

    print('Generating Volatility Adjusted Trend Signal!!')
    ## Get Volatility Adjusted Trend Signal
    df_signal = size_cont.get_volatility_adjusted_trend_signal_continuous(df_trend, ticker_list, volatility_window,
                                                                          annual_trading_days)

In [16]:
end_date = datetime.datetime.now().date()
start_date = end_date - pd.Timedelta(days=cfg['run']['warmup_days'])

# Build kwargs directly from cfg sections
sig_kwargs = {
    # run / universe / data
    "start_date": start_date,
    "end_date": end_date,
    "ticker_list": cfg["universe"]["tickers"],
    "moving_avg_type": cfg["data"]["moving_avg_type"],
    "price_or_returns_calc": cfg["data"]["price_or_returns_calc"],
    "use_coinbase_data": cfg["data"]["use_coinbase_data"],
    "use_saved_files": False,
    "saved_file_end_date": cfg["data"]["saved_file_end_date"],  # convert to date if your fn expects it

    # signals → moving average
    "fast_mavg": cfg["signals"]["moving_average"]["fast_mavg"],
    "slow_mavg": cfg["signals"]["moving_average"]["slow_mavg"],
    "mavg_stepsize": cfg["signals"]["moving_average"]["mavg_stepsize"],
    "mavg_z_score_window": cfg["signals"]["moving_average"]["mavg_z_score_window"],

    # signals → donchian
    "entry_rolling_donchian_window": cfg["signals"]["donchian"]["entry_rolling_donchian_window"],
    "exit_rolling_donchian_window": cfg["signals"]["donchian"]["exit_rolling_donchian_window"],
    "use_donchian_exit_gate": cfg["signals"]["donchian"]["use_donchian_exit_gate"],

    # signals → weighting
    "ma_crossover_signal_weight": cfg["signals"]["weighting"]["ma_crossover_signal_weight"],
    "donchian_signal_weight": cfg["signals"]["weighting"]["donchian_signal_weight"],
    "weighted_signal_ewm_window": cfg["signals"]["weighting"]["weighted_signal_ewm_window"],

    # signals → activation
    "use_activation": cfg["signals"]["activation"]["use_activation"],
    "tanh_activation_constant_dict": cfg["signals"]["activation"]["tanh_activation_constant_dict"],

    # signals → filters → rolling_r2
    "rolling_r2_window": cfg["signals"]["filters"]["rolling_r2"]["rolling_r2_window"],
    "lower_r_sqr_limit": cfg["signals"]["filters"]["rolling_r2"]["lower_r_sqr_limit"],
    "upper_r_sqr_limit": cfg["signals"]["filters"]["rolling_r2"]["upper_r_sqr_limit"],
    "r2_smooth_window": cfg["signals"]["filters"]["rolling_r2"]["r2_smooth_window"],
    "r2_confirm_days": cfg["signals"]["filters"]["rolling_r2"]["r2_confirm_days"],
    "r2_strong_threshold": cfg["signals"]["filters"]["rolling_r2"]["r2_strong_threshold"],

    # signals → filters → vol_of_vol
    "log_std_window": cfg["signals"]["filters"]["vol_of_vol"]["log_std_window"],
    "coef_of_variation_window": cfg["signals"]["filters"]["vol_of_vol"]["coef_of_variation_window"],
    "vol_of_vol_z_score_window": cfg["signals"]["filters"]["vol_of_vol"]["vol_of_vol_z_score_window"],
    "vol_of_vol_p_min": cfg["signals"]["filters"]["vol_of_vol"]["vol_of_vol_p_min"],

    # run
    "long_only": cfg["run"]["long_only"],
}

df_trend = get_trend_donchian_signal_for_portfolio_with_rolling_r_sqr_vol_of_vol(**sig_kwargs)

print('Generating Volatility Adjusted Trend Signal!!')
## Get Volatility Adjusted Trend Signal
df_signal = size_cont.get_volatility_adjusted_trend_signal_continuous(df_trend, ticker_list=cfg['universe']['tickers'],
                                                                      volatility_window=cfg['risk_and_sizing']['volatility_window'],
                                                                      annual_trading_days=cfg['run']['annual_trading_days'])

print('Getting Average True Range for Stop Loss Calculation!!')
## Get Average True Range for Stop Loss Calculation
atr_kwargs = {
    # run / universe / data
    "start_date": start_date,
    "end_date": end_date,
    "ticker_list": cfg["universe"]["tickers"],

    # risk and sizing
    "rolling_atr_window": cfg['risk_and_sizing']['rolling_atr_window'],

    # data
    "price_or_returns_calc": cfg['data']['price_or_returns_calc'],
    "use_coinbase_data": cfg['data']['use_coinbase_data'],
    "use_saved_files": False,
    "saved_file_end_date": cfg['data']['saved_file_end_date']
}
df_atr = size_cont.get_average_true_range_portfolio(**atr_kwargs)
df_signal = pd.merge(df_signal, df_atr, left_index=True, right_index=True, how='left')

Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!


In [28]:
df_signal.head()

,BTC-USD_close,BTC-USD_open,BTC-USD_t_1_close,BTC-USD_t_1_close_pct_returns,BTC-USD_final_signal,BTC-USD_final_weighted_additive_signal,BTC-USD_annualized_volatility_30,BTC-USD_vol_adjusted_trend_signal,ETH-USD_close,ETH-USD_open,ETH-USD_t_1_close,ETH-USD_t_1_close_pct_returns,ETH-USD_final_signal,ETH-USD_final_weighted_additive_signal,ETH-USD_annualized_volatility_30,ETH-USD_vol_adjusted_trend_signal,SOL-USD_close,SOL-USD_open,SOL-USD_t_1_close,SOL-USD_t_1_close_pct_returns,SOL-USD_final_signal,SOL-USD_final_weighted_additive_signal,SOL-USD_annualized_volatility_30,SOL-USD_vol_adjusted_trend_signal,ADA-USD_close,ADA-USD_open,ADA-USD_t_1_close,ADA-USD_t_1_close_pct_returns,ADA-USD_final_signal,ADA-USD_final_weighted_additive_signal,ADA-USD_annualized_volatility_30,ADA-USD_vol_adjusted_trend_signal,AVAX-USD_close,AVAX-USD_open,AVAX-USD_t_1_close,AVAX-USD_t_1_close_pct_returns,AVAX-USD_final_signal,AVAX-USD_final_weighted_additive_signal,AVAX-USD_annualized_volatility_30,AVAX-USD_vol_adjusted_trend_signal,BTC-USD_20_avg_true_range_price,ETH-USD_20_avg_true_range_price,SOL-USD_20_avg_true_range_price,ADA-USD_20_avg_true_range_price,AVAX-USD_20_avg_true_range_price
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-11-09,76716.06,76566.72,NaN,NaN,NaN,NaN,NaN,0.0,3127.97,2964.09,NaN,NaN,NaN,NaN,NaN,0.0,200.00,200.00,NaN,NaN,NaN,NaN,NaN,0.0,0.4965,0.4451,NaN,NaN,NaN,NaN,NaN,0.0,30.39,28.81,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2024-11-10,80428.92,76716.06,76716.06,NaN,NaN,NaN,NaN,0.0,3186.07,3127.96,3127.97,NaN,NaN,NaN,NaN,0.0,210.22,200.00,200.00,NaN,NaN,NaN,NaN,0.0,0.5902,0.4972,0.4965,NaN,NaN,NaN,NaN,0.0,31.91,30.40,30.39,NaN,NaN,NaN,NaN,0.0,1194.090000,202.030000,8.440000,0.071000,2.000000
2024-11-11,88770.73,80427.69,80428.92,0.048397,NaN,NaN,NaN,0.0,3375.78,3186.03,3186.07,0.018574,NaN,NaN,NaN,0.0,222.42,210.22,210.22,0.051100,NaN,NaN,NaN,0.0,0.6139,0.5909,0.5902,0.188721,NaN,NaN,NaN,0.0,35.75,31.92,31.91,0.050016,NaN,NaN,NaN,0.0,1556.457619,200.128095,9.240000,0.083457,2.157143
2024-11-12,88035.43,88770.74,88770.73,0.103717,NaN,NaN,NaN,0.0,3247.21,3375.79,3375.78,0.059544,NaN,NaN,NaN,0.0,211.87,222.41,222.42,0.058034,NaN,NaN,NaN,0.0,0.5715,0.6142,0.6139,0.040156,NaN,NaN,NaN,0.0,33.96,35.77,35.75,0.120338,NaN,NaN,NaN,0.0,2302.381655,208.069229,10.329524,0.081261,2.437415
2024-11-13,90493.82,88035.43,88035.43,-0.008283,NaN,NaN,NaN,0.0,3191.45,3247.54,3247.21,-0.038086,NaN,NaN,NaN,0.0,215.46,211.88,211.87,-0.047433,NaN,NaN,NaN,0.0,0.5791,0.5716,0.5715,-0.069067,NaN,NaN,NaN,0.0,33.17,33.98,33.96,-0.050070,NaN,NaN,NaN,0.0,2567.869117,210.768350,11.285760,0.082989,2.686233


In [66]:
def get_target_notional_by_ticker(df, ticker_list, initial_capital, rolling_cov_window,
                                  rolling_atr_window, atr_multiplier, cash_buffer_percentage,
                                  annualized_target_volatility, transaction_cost_est=0.001,
                                  passive_trade_rate=0.05, notional_threshold_pct=0.02,
                                  min_trade_notional_abs=10, cooldown_counter_threshold=3,
                                  annual_trading_days=365, use_specific_start_date=False,
                                  signal_start_date=None):

    ## Calculate the covariance matrix for tickers in the portfolio
    returns_cols = [f'{ticker}_t_1_close_pct_returns' for ticker in ticker_list]
    cov_matrix = df[returns_cols].rolling(rolling_cov_window).cov(pairwise=True).dropna()

    ## Delete rows prior to the first available date of the covariance matrix
    cov_matrix_start_date = cov_matrix.index[0][0]
    df = df[df.index >= cov_matrix_start_date]

    ## Derive the Daily Target Portfolio Volatility
    daily_target_volatility = annualized_target_volatility / np.sqrt(annual_trading_days)

    ## Reorder dataframe columns
    for ticker in ticker_list:
        df[f'{ticker}_new_position_size'] = 0.0
        df[f'{ticker}_new_position_notional'] = 0.0
        df[f'{ticker}_open_position_size'] = 0.0
        df[f'{ticker}_open_position_notional'] = 0.0
        df[f'{ticker}_actual_position_size'] = 0.0
        df[f'{ticker}_actual_position_notional'] = 0.0
        df[f'{ticker}_short_sale_proceeds'] = 0.0
        df[f'{ticker}_new_position_entry_exit_price'] = 0.0
        df[f'{ticker}_target_vol_normalized_weight'] = 0.0
        df[f'{ticker}_target_notional'] = 0.0
        df[f'{ticker}_target_size'] = 0.0
        df[f'{ticker}_cash_shrink_factor'] = 0.0
        df[f'{ticker}_stop_loss'] = 0.0
        df[f'{ticker}_stopout_flag'] = False
        df[f'{ticker}_cooldown_counter'] = 0.0
        df[f'{ticker}_event'] = np.nan
    ord_cols = size_bin.reorder_columns_by_ticker(df.columns, ticker_list)
    df = df[ord_cols]

    ## Portfolio Level Cash and Positions are all set to 0
    df['daily_portfolio_volatility'] = 0.0
    df['available_cash'] = 0.0
    df['count_of_positions'] = 0.0
    df['total_actual_position_notional'] = 0.0
    df['total_target_notional'] = 0.0
    df['total_portfolio_value'] = 0.0
    df['total_portfolio_value_upper_limit'] = 0.0
    df['target_vol_scaling_factor'] = 1.0
    df['cash_scaling_factor'] = 1.0
    df['final_scaling_factor'] = 1.0

    ## Cash and the Total Portfolio Value on Day 1 is the initial capital for the strategy
    if use_specific_start_date:
        start_index_position = df.index.get_loc(signal_start_date)
    else:
        start_index_position = 0
    df['available_cash'][start_index_position] = initial_capital
    df['total_portfolio_value'][start_index_position] = initial_capital

    ## Identify Daily Positions starting from day 2
    for date in df.index[start_index_position + 1:]:
        previous_date = df.index[df.index.get_loc(date) - 1]

        ## Start the day with the available cash from yesterday
        df['available_cash'].loc[date] = df['available_cash'].loc[previous_date]

        ## Roll Portfolio Value from the Previous Day
        df['total_portfolio_value'].loc[date] = df['total_portfolio_value'].loc[previous_date]

        ## Update Total Portfolio Value Upper Limit based on the Total Portfolio Value
        total_portfolio_value_upper_limit = (df['total_portfolio_value'].loc[date] *
                                             (1 - cash_buffer_percentage))
        df['total_portfolio_value_upper_limit'].loc[date] = total_portfolio_value_upper_limit

        ## Calculate the target notional by ticker
        df = size_cont.get_target_volatility_position_sizing(df, cov_matrix, date, ticker_list, daily_target_volatility,
                                                             total_portfolio_value_upper_limit)

        ## Adjust Positions for Cash Available
        desired_positions, cash_shrink_factor = size_cont.get_cash_adjusted_desired_positions(
            df, date, previous_date, ticker_list, cash_buffer_percentage, transaction_cost_est, passive_trade_rate,
            notional_threshold_pct, min_trade_notional_abs)

    ## Apply Cash Shrink Factor to Desired Positions
    for ticker in ticker_list:
        new_trade_notional = desired_positions[ticker]['new_trade_notional'] * cash_shrink_factor
        trade_fees = desired_positions[ticker]['trade_fees'] * cash_shrink_factor

    return df, desired_positions

In [68]:
df_final, desired_positions = get_target_notional_by_ticker(**target_vol_kwargs)

In [70]:
df_final.head()

,BTC-USD_20_avg_true_range_price,BTC-USD_actual_position_notional,BTC-USD_actual_position_size,BTC-USD_annualized_volatility_30,BTC-USD_cash_shrink_factor,BTC-USD_close,BTC-USD_cooldown_counter,BTC-USD_event,BTC-USD_final_signal,BTC-USD_final_weighted_additive_signal,BTC-USD_new_position_entry_exit_price,BTC-USD_new_position_notional,BTC-USD_new_position_size,BTC-USD_open,BTC-USD_open_position_notional,BTC-USD_open_position_size,BTC-USD_short_sale_proceeds,BTC-USD_stop_loss,BTC-USD_stopout_flag,BTC-USD_t_1_close,BTC-USD_t_1_close_pct_returns,BTC-USD_target_notional,BTC-USD_target_size,BTC-USD_target_vol_normalized_weight,BTC-USD_vol_adjusted_trend_signal,ETH-USD_20_avg_true_range_price,ETH-USD_actual_position_notional,ETH-USD_actual_position_size,ETH-USD_annualized_volatility_30,ETH-USD_cash_shrink_factor,ETH-USD_close,ETH-USD_cooldown_counter,ETH-USD_event,ETH-USD_final_signal,ETH-USD_final_weighted_additive_signal,ETH-USD_new_position_entry_exit_price,ETH-USD_new_position_notional,ETH-USD_new_position_size,ETH-USD_open,ETH-USD_open_position_notional,ETH-USD_open_position_size,ETH-USD_short_sale_proceeds,ETH-USD_stop_loss,ETH-USD_stopout_flag,ETH-USD_t_1_close,ETH-USD_t_1_close_pct_returns,ETH-USD_target_notional,ETH-USD_target_size,ETH-USD_target_vol_normalized_weight,ETH-USD_vol_adjusted_trend_signal,SOL-USD_20_avg_true_range_price,SOL-USD_actual_position_notional,SOL-USD_actual_position_size,SOL-USD_annualized_volatility_30,SOL-USD_cash_shrink_factor,SOL-USD_close,SOL-USD_cooldown_counter,SOL-USD_event,SOL-USD_final_signal,SOL-USD_final_weighted_additive_signal,SOL-USD_new_position_entry_exit_price,SOL-USD_new_position_notional,SOL-USD_new_position_size,SOL-USD_open,SOL-USD_open_position_notional,SOL-USD_open_position_size,SOL-USD_short_sale_proceeds,SOL-USD_stop_loss,SOL-USD_stopout_flag,SOL-USD_t_1_close,SOL-USD_t_1_close_pct_returns,SOL-USD_target_notional,SOL-USD_target_size,SOL-USD_target_vol_normalized_weight,SOL-USD_vol_adjusted_trend_signal,ADA-USD_20_avg_true_range_price,ADA-USD_actual_position_notional,ADA-USD_actual_position_size,ADA-USD_annualized_volatility_30,ADA-USD_cash_shrink_factor,ADA-USD_close,ADA-USD_cooldown_counter,ADA-USD_event,ADA-USD_final_signal,ADA-USD_final_weighted_additive_signal,ADA-USD_new_position_entry_exit_price,ADA-USD_new_position_notional,ADA-USD_new_position_size,ADA-USD_open,ADA-USD_open_position_notional,ADA-USD_open_position_size,ADA-USD_short_sale_proceeds,ADA-USD_stop_loss,ADA-USD_stopout_flag,ADA-USD_t_1_close,ADA-USD_t_1_close_pct_returns,ADA-USD_target_notional,ADA-USD_target_size,ADA-USD_target_vol_normalized_weight,ADA-USD_vol_adjusted_trend_signal,AVAX-USD_20_avg_true_range_price,AVAX-USD_actual_position_notional,AVAX-USD_actual_position_size,AVAX-USD_annualized_volatility_30,AVAX-USD_cash_shrink_factor,AVAX-USD_close,AVAX-USD_cooldown_counter,AVAX-USD_event,AVAX-USD_final_signal,AVAX-USD_final_weighted_additive_signal,AVAX-USD_new_position_entry_exit_price,AVAX-USD_new_position_notional,AVAX-USD_new_position_size,AVAX-USD_open,AVAX-USD_open_position_notional,AVAX-USD_open_position_size,AVAX-USD_short_sale_proceeds,AVAX-USD_stop_loss,AVAX-USD_stopout_flag,AVAX-USD_t_1_close,AVAX-USD_t_1_close_pct_returns,AVAX-USD_target_notional,AVAX-USD_target_size,AVAX-USD_target_vol_normalized_weight,AVAX-USD_vol_adjusted_trend_signal,daily_portfolio_volatility,available_cash,count_of_positions,total_actual_position_notional,total_target_notional,total_portfolio_value,total_portfolio_value_upper_limit,target_vol_scaling_factor,cash_scaling_factor,final_scaling_factor,cash_shrink_factor
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-11-30,3571.214469,0.0,0.0,NaN,0.0,96465.42,0.0,NaN,NaN,NaN,0.0,0.0,0.0,97489.80,0.0,0.0,0.0,0.0,False,97490.56,0.019077,0.0,0.0,0.0,0.0,198.873255,0.0,0.0,NaN,0.0,3706.35,0.0,NaN,NaN,NaN,0.0,0.0,0.0,3593.38,0.0,0.0,0.0,0.0,False,3593.67,0.004068,0.0,0.0,0.0,0.0,

In [72]:
desired_positions

{'BTC-USD': {'new_trade_notional': 0, 'trade_fees': 0},
 'ETH-USD': {'new_trade_notional': 5006.903890841127,
  'trade_fees': 63.58767941368231},
 'SOL-USD': {'new_trade_notional': 2692.3722783016624,
  'trade_fees': 34.19312793443111},
 'ADA-USD': {'new_trade_notional': 763.3374008212526,
  'trade_fees': 9.694384990429906},
 'AVAX-USD': {'new_trade_notional': 0, 'trade_fees': 0}}

In [48]:
cash_shrink_factor

1.0

In [62]:
## Calculate the covariance matrix for tickers in the portfolio
returns_cols = [f'{ticker}_t_1_close_pct_returns' for ticker in cfg['universe']['tickers']]
cov_matrix = df[returns_cols].rolling(cfg['risk_and_sizing']['rolling_cov_window']).cov(pairwise=True).dropna()

In [64]:
cov_matrix

BTC-USD_t_1_close_pct_returns  \
date                                                                       
2024-12-19 BTC-USD_t_1_close_pct_returns                        0.000588   
           ETH-USD_t_1_close_pct_returns                        0.000734   
           SOL-USD_t_1_close_pct_returns                        0.000594   
           ADA-USD_t_1_close_pct_returns                        0.000866   
           AVAX-USD_t_1_close_pct_returns                       0.000929   
2024-12-20 BTC-USD_t_1_close_pct_returns                        0.000616   
           ETH-USD_t_1_close_pct_returns                        0.000818   
           SOL-USD_t_1_close_pct_returns                        0.000645   
           ADA-USD_t_1_close_pct_returns                        0.000953   
           AVAX-USD_t_1_close_pct_returns                       0.001022   
2024-12-21 BTC-USD_t_1_close_pct_returns                        0.000610   
           ETH-USD_t_1_close_pct_returns                        0.000841   
           SOL-USD_t_1_close_pct_returns                        0.000639   
           ADA-USD_t_1_close_pct_returns                        0.000975   
           AVAX-USD_t_1_close_pct_returns                       0.001029   
2024-12-22 BTC-USD_t_1_close_pct_returns                        0.000609   
           ETH-USD_t_1_close_pct_returns                        0.000851   
           SOL-USD_t_1_close_pct_returns                        0.000654   
           ADA-USD_t_1_close_pct_returns                        0.000958   
           AVAX-USD_t_1_close_pct_returns                       0.001043   
2024-12-23 BTC-USD_t_1_close_pct_returns                        0.000624   
           ETH-USD_t_1_close_pct_returns                        0.000856   
           SOL-USD_t_1_close_pct_returns                        0.000620   
           ADA-USD_t_1_close_pct_returns                        0.001016   
           AVAX-USD_t_1_close_pct_returns                       0.001186   
2024-12-24 BTC-USD_t_1_close_pct_returns                        0.000624   
           ETH-USD_t_1_close_pct_returns                        0.000848   
           SOL-USD_t_1_close_pct_returns                        0.000608   
           ADA-USD_t_1_close_pct_returns                        0.001006   
           AVAX-USD_t_1_close_pct_returns                       0.001173   
2024-12-25 BTC-USD_t_1_close_pct_returns                        0.000664   
           ETH-USD_t_1_close_pct_returns                        0.000799   
           SOL-USD_t_1_close_pct_returns                        0.000722   
           ADA-USD_t_1_close_pct_returns                        0.001040   
           AVAX-USD_t_1_close_pct_returns                       0.001239   
2024-12-26 BTC-USD_t_1_close_pct_returns                        0.000650   
           ETH-USD_t_1_close_pct_returns                        0.000790   
           SOL-USD_t_1_close_pct_returns                        0.000760   
           ADA-USD_t_1_close_pct_returns                        0.001024   
           AVAX-USD_t_1_close_pct_returns                       0.001197   
2024-12-27 BTC-USD_t_1_close_pct_returns                        0.000675   
           ETH-USD_t_1_close_pct_returns                        0.000772   
           SOL-USD_t_1_close_pct_returns                        0.000809   
           ADA-USD_t_1_close_pct_returns                        0.001007   
           AVAX-USD_t_1_close_pct_returns                       0.001229   
2024-12-28 BTC-USD_t_1_close_pct_returns                        0.000684   
           ETH-USD_t_1_close_pct_returns                        0.000765   
           SOL-USD_t_1_close_pct_returns                        0.000815   
           ADA-USD_t_1_close_pct_returns                        0.000984   
           AVAX-USD_t_1_close_pct_returns                       0.001234   
2024-12-29 BTC-USD_t_1_close_pct_returns                        0.000681   
           ETH-

In [74]:
cn_client = cn.get_coinbase_rest_api_client(cn.key_location)
df_portfolio_positions = cn.get_portfolio_breakdown(cn_client)

In [86]:
[f'{i[:-4]}' for i in cfg['universe']['tickers']]

['BTC', 'ETH', 'SOL', 'ADA', 'AVAX']

In [88]:
df_portfolio_positions[df_portfolio_positions.asset.isin([f'{i[:-4]}' for i in cfg['universe']['tickers']])]#.head()

,asset,account_uuid,asset_uuid,total_balance_fiat,available_to_trade_fiat,total_balance_crypto,allocation,cost_basis_value,cost_basis_currency,is_cash,average_entry_price_value,average_entry_price_currency,available_to_trade_crypto,unrealized_pnl,available_to_transfer_fiat,available_to_transfer_crpyto
1,AVAX,0d4dc05a-5e31-5199-9b62-ac2df617e922,9d06e463-b3ba-5abf-9082-8761846b28ab,276.4045,276.4045,11.440584,0.011211,850.000000000000000103759628,USD,False,74.2969067813213269,USD,11.440584,-573.59550,276.4045,11.440584
7,ETH,40a601d9-8ffa-500d-8f02-66d9123a2514,d85dce9b-5b73-5c3c-8978-522ce1d1c1b4,10915.0880,10915.0880,2.538363,0.442714,3960.700290797639258827167594,USD,False,1560.3364108209971221,USD,2.538363,6954.38770,10915.0880,2.538363
22,ADA,b03d98bb-9471-503d-ada1-e22a782bb509,63062039-7afb-56ff-8e19-5e3215dc404a,1015.2610,1015.2610,1235.561600,0.041179,1350.5450000000000114567865,USD,False,1.0930616596669401,USD,1235.561600,-335.28406,1015.2610,1235.561600
26,BTC,dd9f67c8-a344-55e1-8695-2d4e3a4e77e5,5b71fc48-3dd3-540c-809b-f8c94d0e68b5,5612.1094,5612.1094,0.050673,0.227626,1429.8729299999999999994292,USD,False,28217.8105335199163415,USD,0.050673,4182.23630,5612.1094,0.050673
27,SOL,e1475725-5f00-5746-bc35-1225b21849ca,4f039497-3af8-5bb3-951c-6df9afa9be1c,4134.1470,4134.1470,20.376297,0.167680,1433.1950000000000000083600982,USD,False,70.3363796989895848,USD,20.376297,2700.95200,4134.1470,20.376297


In [17]:
print('Calculating Volatility Targeted Position Size and Cash Management!!')
## Get Target Volatility Position Sizing and Run Cash Management
target_vol_kwargs = {
    "df": df_signal,
    "ticker_list": cfg['universe']['tickers'],
    "initial_capital": cfg['run']['initial_capital'],
    "rolling_cov_window": cfg['risk_and_sizing']['rolling_cov_window'],
    "rolling_atr_window": cfg['risk_and_sizing']['rolling_atr_window'],
    "atr_multiplier": cfg['risk_and_sizing']['atr_multiplier'],
    "cash_buffer_percentage": cfg['risk_and_sizing']['cash_buffer_percentage'],
    "annualized_target_volatility": cfg['risk_and_sizing']['annualized_target_volatility'],
    "transaction_cost_est": cfg['execution_and_costs']['transaction_cost_est'],
    "passive_trade_rate": cfg['execution_and_costs']['passive_trade_rate'],
    "notional_threshold_pct": cfg['execution_and_costs']['notional_threshold_pct'],
    "min_trade_notional_abs": cfg['execution_and_costs']['min_trade_notional_abs'],
    "cooldown_counter_threshold": cfg['execution_and_costs']['cooldown_counter_threshold'],
    "annual_trading_days": cfg['run']['annual_trading_days'],
    "use_specific_start_date": False,
    "signal_start_date": cfg['run']['signal_start_date']
}
df = size_cont.get_target_volatility_daily_portfolio_positions(**target_vol_kwargs)
# df = size_cont.get_target_volatility_daily_portfolio_positions(
#     df_signal, ticker_list=cfg['universe']['tickers'], initial_capital=cfg['run']['initial_capital'], rolling_cov_window=cfg['risk_and_sizing']['rolling_cov_window'],
#     rolling_atr_window=cfg['risk_and_sizing']['rolling_atr_window'], atr_multiplier=cfg['risk_and_sizing']['atr_multiplier'],
#     cash_buffer_percentage=cfg['risk_and_sizing']['cash_buffer_percentage'], annualized_target_volatility=cfg['risk_and_sizing']['annualized_target_volatility'],
#     transaction_cost_est=cfg['execution_and_costs']['transaction_cost_est'], passive_trade_rate=cfg['execution_and_costs']['passive_trade_rate'],
#     notional_threshold_pct=cfg['execution_and_costs']['notional_threshold_pct'], cooldown_counter_threshold=cfg['execution_and_costs']['cooldown_counter_threshold'],
#     annual_trading_days=cfg['run']['annual_trading_days'], use_specific_start_date=False,
#     signal_start_date=start_date)

Calculating Volatility Targeted Position Size and Cash Management!!


In [ ]:
df_signal.tail()

In [20]:
df.tail()

,BTC-USD_20_avg_true_range_price,BTC-USD_actual_position_notional,BTC-USD_actual_position_size,BTC-USD_annualized_volatility_30,BTC-USD_cash_shrink_factor,BTC-USD_close,BTC-USD_cooldown_counter,BTC-USD_event,BTC-USD_final_signal,BTC-USD_final_weighted_additive_signal,BTC-USD_new_position_entry_exit_price,BTC-USD_new_position_notional,BTC-USD_new_position_size,BTC-USD_open,BTC-USD_open_position_notional,BTC-USD_open_position_size,BTC-USD_short_sale_proceeds,BTC-USD_stop_loss,BTC-USD_stopout_flag,BTC-USD_t_1_close,BTC-USD_t_1_close_pct_returns,BTC-USD_target_notional,BTC-USD_target_size,BTC-USD_target_vol_normalized_weight,BTC-USD_vol_adjusted_trend_signal,ETH-USD_20_avg_true_range_price,ETH-USD_actual_position_notional,ETH-USD_actual_position_size,ETH-USD_annualized_volatility_30,ETH-USD_cash_shrink_factor,ETH-USD_close,ETH-USD_cooldown_counter,ETH-USD_event,ETH-USD_final_signal,ETH-USD_final_weighted_additive_signal,ETH-USD_new_position_entry_exit_price,ETH-USD_new_position_notional,ETH-USD_new_position_size,ETH-USD_open,ETH-USD_open_position_notional,ETH-USD_open_position_size,ETH-USD_short_sale_proceeds,ETH-USD_stop_loss,ETH-USD_stopout_flag,ETH-USD_t_1_close,ETH-USD_t_1_close_pct_returns,ETH-USD_target_notional,ETH-USD_target_size,ETH-USD_target_vol_normalized_weight,ETH-USD_vol_adjusted_trend_signal,SOL-USD_20_avg_true_range_price,SOL-USD_actual_position_notional,SOL-USD_actual_position_size,SOL-USD_annualized_volatility_30,SOL-USD_cash_shrink_factor,SOL-USD_close,SOL-USD_cooldown_counter,SOL-USD_event,SOL-USD_final_signal,SOL-USD_final_weighted_additive_signal,SOL-USD_new_position_entry_exit_price,SOL-USD_new_position_notional,SOL-USD_new_position_size,SOL-USD_open,SOL-USD_open_position_notional,SOL-USD_open_position_size,SOL-USD_short_sale_proceeds,SOL-USD_stop_loss,SOL-USD_stopout_flag,SOL-USD_t_1_close,SOL-USD_t_1_close_pct_returns,SOL-USD_target_notional,SOL-USD_target_size,SOL-USD_target_vol_normalized_weight,SOL-USD_vol_adjusted_trend_signal,ADA-USD_20_avg_true_range_price,ADA-USD_actual_position_notional,ADA-USD_actual_position_size,ADA-USD_annualized_volatility_30,ADA-USD_cash_shrink_factor,ADA-USD_close,ADA-USD_cooldown_counter,ADA-USD_event,ADA-USD_final_signal,ADA-USD_final_weighted_additive_signal,ADA-USD_new_position_entry_exit_price,ADA-USD_new_position_notional,ADA-USD_new_position_size,ADA-USD_open,ADA-USD_open_position_notional,ADA-USD_open_position_size,ADA-USD_short_sale_proceeds,ADA-USD_stop_loss,ADA-USD_stopout_flag,ADA-USD_t_1_close,ADA-USD_t_1_close_pct_returns,ADA-USD_target_notional,ADA-USD_target_size,ADA-USD_target_vol_normalized_weight,ADA-USD_vol_adjusted_trend_signal,AVAX-USD_20_avg_true_range_price,AVAX-USD_actual_position_notional,AVAX-USD_actual_position_size,AVAX-USD_annualized_volatility_30,AVAX-USD_cash_shrink_factor,AVAX-USD_close,AVAX-USD_cooldown_counter,AVAX-USD_event,AVAX-USD_final_signal,AVAX-USD_final_weighted_additive_signal,AVAX-USD_new_position_entry_exit_price,AVAX-USD_new_position_notional,AVAX-USD_new_position_size,AVAX-USD_open,AVAX-USD_open_position_notional,AVAX-USD_open_position_size,AVAX-USD_short_sale_proceeds,AVAX-USD_stop_loss,AVAX-USD_stopout_flag,AVAX-USD_t_1_close,AVAX-USD_t_1_close_pct_returns,AVAX-USD_target_notional,AVAX-USD_target_size,AVAX-USD_target_vol_normalized_weight,AVAX-USD_vol_adjusted_trend_signal,daily_portfolio_volatility,available_cash,count_of_positions,total_actual_position_notional,total_target_notional,total_portfolio_value,total_portfolio_value_upper_limit,target_vol_scaling_factor,cash_scaling_factor,final_scaling_factor,cash_shrink_factor
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-09-01,3081.499094,0.0,0.0,0.357586,0.0,109240.55,0.0,No Position,0.0,-0.278088,108247.95,0.0,0.0,108247.95,0.0,0.0,0.0,0.0,False,108247.95,-0.005329,0.0,0.0,0.0,0.0,244.279296,9748.073449,2.219557,0.908245,0.0,4314.76,0.0,Trim Long Position,0.335957,0.404541,439

In [ ]:
df.tail()

In [ ]:
df['AVAX-USD_event']

In [24]:
df[target_notional_cols]

,BTC-USD_target_notional,ETH-USD_target_notional,SOL-USD_target_notional,ADA-USD_target_notional,AVAX-USD_target_notional
date,,,,,
2024-11-30,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2024-12-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2024-12-02,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2024-12-03,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2024-12-04,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2024-12-05,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2024-12-06,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2024-12-07,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2024-12-08,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
df.iloc[-1]

In [ ]:
cfg

In [22]:
signal_cols = [f'{ticker}_final_signal' for ticker in cfg["universe"]["tickers"]]
vol_adj_signal_cols = [f'{ticker}_vol_adjusted_trend_signal' for ticker in cfg["universe"]["tickers"]]
target_notional_cols = [f'{ticker}_target_notional' for ticker in cfg["universe"]["tickers"]]
df_trend[signal_cols]

,BTC-USD_final_signal,ETH-USD_final_signal,SOL-USD_final_signal,ADA-USD_final_signal,AVAX-USD_final_signal
date,,,,,
2024-11-09,NaN,NaN,NaN,NaN,NaN
2024-11-10,NaN,NaN,NaN,NaN,NaN
2024-11-11,NaN,NaN,NaN,NaN,NaN
2024-11-12,NaN,NaN,NaN,NaN,NaN
2024-11-13,NaN,NaN,NaN,NaN,NaN
2024-11-14,NaN,NaN,NaN,NaN,NaN
2024-11-15,NaN,NaN,NaN,NaN,NaN
2024-11-16,NaN,NaN,NaN,NaN,NaN
2024-11-17,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_trend[vol_adj_signal_cols]

In [ ]:
def apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol(
        start_date, end_date, ticker_list, fast_mavg, slow_mavg, mavg_stepsize, mavg_z_score_window,
        entry_rolling_donchian_window, exit_rolling_donchian_window, use_donchian_exit_gate,
        ma_crossover_signal_weight, donchian_signal_weight, weighted_signal_ewm_window,
        rolling_r2_window=30, lower_r_sqr_limit=0.2, upper_r_sqr_limit=0.8, r2_smooth_window=3, r2_confirm_days=0,
        log_std_window=14, coef_of_variation_window=30, vol_of_vol_z_score_window=252, vol_of_vol_p_min=0.6,
        r2_strong_threshold=0.8, use_activation=True, tanh_activation_constant_dict=None, moving_avg_type='exponential',
        long_only=False, price_or_returns_calc='price', initial_capital=15000, rolling_cov_window=20,
        volatility_window=20, rolling_atr_window=20, atr_multiplier=0.5, transaction_cost_est=0.001,
        passive_trade_rate=0.05, notional_threshold_pct=0.05, cooldown_counter_threshold=3, use_coinbase_data=True,
        use_saved_files=True, saved_file_end_date='2025-07-31', rolling_sharpe_window=50, cash_buffer_percentage=0.10,
        annualized_target_volatility=0.20, annual_trading_days=365, use_specific_start_date=False,
        signal_start_date=None):

    ## Check if data is available for all the tickers
    date_list = cn.coinbase_start_date_by_ticker_dict
    ticker_list = [ticker for ticker in ticker_list if pd.Timestamp(date_list[ticker]).date() < end_date]

    print('Generating Moving Average Ribbon Signal!!')
    ## Generate Trend Signal for all tickers

    df_trend = get_trend_donchian_signal_for_portfolio_with_rolling_r_sqr_vol_of_vol(
        start_date=start_date, end_date=end_date, ticker_list=ticker_list, fast_mavg=fast_mavg, slow_mavg=slow_mavg,
        mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window,
        entry_rolling_donchian_window=entry_rolling_donchian_window,
        exit_rolling_donchian_window=exit_rolling_donchian_window, use_donchian_exit_gate=use_donchian_exit_gate,
        ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight,
        weighted_signal_ewm_window=weighted_signal_ewm_window, rolling_r2_window=rolling_r2_window,
        lower_r_sqr_limit=lower_r_sqr_limit, upper_r_sqr_limit=upper_r_sqr_limit, r2_smooth_window=r2_smooth_window,
        r2_confirm_days=r2_confirm_days, log_std_window=log_std_window, coef_of_variation_window=coef_of_variation_window,
        vol_of_vol_z_score_window=vol_of_vol_z_score_window, vol_of_vol_p_min=vol_of_vol_p_min,
        r2_strong_threshold=r2_strong_threshold, use_activation=use_activation,
        tanh_activation_constant_dict=tanh_activation_constant_dict, moving_avg_type=moving_avg_type,
        long_only=long_only, price_or_returns_calc=price_or_returns_calc, use_coinbase_data=use_coinbase_data,
        use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date)

    print('Generating Volatility Adjusted Trend Signal!!')
    ## Get Volatility Adjusted Trend Signal
    df_signal = size_cont.get_volatility_adjusted_trend_signal_continuous(df_trend, ticker_list, volatility_window,
                                                                          annual_trading_days)

    print('Getting Average True Range for Stop Loss Calculation!!')
    ## Get Average True Range for Stop Loss Calculation
    df_atr = size_cont.get_average_true_range_portfolio(start_date=start_date, end_date=end_date,
                                                        ticker_list=ticker_list, rolling_atr_window=rolling_atr_window,
                                                        price_or_returns_calc='price',
                                                        use_coinbase_data=use_coinbase_data,
                                                        use_saved_files=use_saved_files,
                                                        saved_file_end_date=saved_file_end_date)
    df_signal = pd.merge(df_signal, df_atr, left_index=True, right_index=True, how='left')

    print('Calculating Volatility Targeted Position Size and Cash Management!!')
    ## Get Target Volatility Position Sizing and Run Cash Management
    df = size_cont.get_target_volatility_daily_portfolio_positions(
        df_signal, ticker_list=ticker_list, initial_capital=initial_capital, rolling_cov_window=rolling_cov_window,
        rolling_atr_window=rolling_atr_window, atr_multiplier=atr_multiplier,
        cash_buffer_percentage=cash_buffer_percentage, annualized_target_volatility=annualized_target_volatility,
        transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate,
        notional_threshold_pct=notional_threshold_pct, cooldown_counter_threshold=cooldown_counter_threshold,
        annual_trading_days=annual_trading_days, use_specific_start_date=use_specific_start_date,
        signal_start_date=signal_start_date)

    print('Calculating Portfolio Performance!!')
    ## Calculate Portfolio Performance
    df = size_bin.calculate_portfolio_returns(df, rolling_sharpe_window)

    return df
